# [Day 1: Inverse Captcha](https://adventofcode.com/2017/day/1)

## Part A

Given a string of integers output the sum of the digits that match the next digit in the list (and the list is circular so the last number counts as adjacent to the first). So "21233142" should be 3 + 2 = 5 (the adjacent 3's in the middle plus the circularly adjacent 2's).

My first solution is a pretty straightforward for loop that takes advantage of Python's negative array indexing to avoid doing anything extra for the circular requirement.

In [1]:
def test_inverse_captcha(f):
    assert(f("1122") == 3)
    assert(f("1111") == 4)
    assert(f("1234") == 0)
    assert(f("91212129") == 9)
    assert(f("") == 0)
    assert(f("3") == 3)

def find_inverse_captcha(digits):
    total = 0
    for i in range(len(digits)):
        if digits[i] == digits[i-1]:
            total += int(digits[i])
    return total

test_inverse_captcha(find_inverse_captcha)

The only thing noticeable about this solution is using the fact that in Python, `array[-1]` refers to the last element of the array (and similarly `array[-i]` refers to the ith last element). So by comparing `digits[i]` with `digits[i-1]` we're able to nicely check the first vs last digit at the `i = 0` step, whereas if we had compared to `digits[i+1]` we would've had to do something extra (e.g using `(i + 1) % N` to make the last index wrap back to the first or appending the first digit to the end of the digit string).

Note also that I could've avoided the `for i in range(len(digits))` with the fancier `for (i, digit) in enumerate(digits)`. `enumerate` on a list returns a list of tuples `(index, element)` on the list; for example:

In [2]:
for (x, y) in enumerate(["a", "b", "c"]):
    print(str(x) + " " + y)

0 a
1 b
2 c


But I personally find `range(len(digits))` simpler and more immediately understandable. However, `enumerate` does allow for a nice one-liner solution:

In [3]:
def find_inverse_captcha_short(digits):
    return sum(int(d) for (i, d) in enumerate(digits) if digits[i] == digits[i-1])

test_inverse_captcha(find_inverse_captcha_short)

Here we use the magic of Python's list comprehensions. We need enumerate for this since here we need to capture both the index (for comparing adjacent digits) and the digit (for outputting the sum) in the same line. If you're not familiar with list comprehension and how the code above is working, hopefully the following examples will clarify:

In [4]:
# sum finds the sum of a list of numbers
sum([1,2,3])

6

In [5]:
# List comprehension lets you construct a list by saying "x for y in list" where x can make use of y,
# which is a variable that will iterate through the values in list
[2*x for x in [1,2,3]]

[2, 4, 6]

In [6]:
# We can also use an if statement in the list comprehension to filter out results
[x*x for x in [1,2,3,4,5,6] if x > 3]

[16, 25, 36]

## Part B

Now instead of comparing adjacent digits, we need to compare against the digit halfway around the circular list. That is, for "21233142" we want to be comparing the first digit (2) against the fifth digit (2). Thankfully we can assume we have an even number of digits. This problem is essentially the same as the previous one, but instead of comparing `i` and `i+1`, we're comparing `i` and `i + len(digits)/2`.

In [7]:
def test_halfway_captcha(f):
    assert(f("1212") == 6)
    assert(f("1221") == 0)
    assert(f("123425") == 4)
    assert(f("123123") == 12)
    assert(f("12131415") == 4)
    assert(f("") == 0)

def find_halfway_captcha(digits):
    assert(len(digits) % 2 == 0)
    step = len(digits) // 2 # Note that // is integer division while / will return a float
    total = 0
    for i in range(len(digits)):
        if digits[i] == digits[i - step]:
            total += int(digits[i])
    return total

def find_halfway_captcha_short(digits):
    assert(len(digits) % 2 == 0)
    return sum(int(d) for (i, d) in enumerate(digits) if digits[i] == digits[i - len(digits)//2])

test_halfway_captcha(find_halfway_captcha)
test_halfway_captcha(find_halfway_captcha_short)

However, there's a nice symmetry we can exploit here because the step is *exactly* halfway. Note that if `i` and `(i + N/2) % N` match, then we automatically know `(i + N/2) % N` and `(i + N/2 + N/2) % N = i` will match. So actually we only have to do the comparisons for the first half of the array, and then double the resulting sum.

In [8]:
def find_halfway_captcha_faster(digits):
    assert(len(digits) % 2 == 0)
    halfLen = len(digits) // 2
    total = 0
    for i in range(halfLen):
        if digits[i] == digits[i - halfLen]:
            total += int(digits[i])
    return 2 * total

test_halfway_captcha(find_halfway_captcha_faster)    

Now finally we'll look at some timing results.

In [9]:
from random import randint

# Random 1000 digit string
big_digits = "".join(str(randint(0,9)) for i in range(1000))

%timeit find_halfway_captcha(big_digits)
%timeit find_halfway_captcha_short(big_digits)
%timeit find_halfway_captcha_faster(big_digits)

118 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
209 µs ± 8.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
56.6 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


As expected, `find_halfway_captcha_faster` is twice as fast as `find_halfway_captcha` since they're essentially the same method but the faster version only checks half the array size. `find_halfway_captcha_short` is significantly (about two times) slower, probably as a result of having to do the `len(digits)//2` calculation for every digit and also possibly because `enumerate()` is a bit slower than `range(len())`.

In [10]:
def find_halfway_captcha_short_2(digits):
    assert(len(digits) % 2 == 0)
    N = len(digits) // 2
    return sum(int(d) for (i, d) in enumerate(digits) if digits[i] == digits[i - N])

%timeit find_halfway_captcha_short_2(big_digits)

132 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


This does indeed seem to be the case, as caching the `len(digits) // 2` value brings the list comprehension method's speed much closer to the for loops.

## Acknowledgements

Thanks to [keyao21](https://github.com/keyao21) for pointing out that I shouldn't name my accumulation variable `sum` since that's a built-in function in Python.